In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import io
import pyarrow.parquet as pq
import pickle

from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA

from statistics import mean
# Optmization
from bayes_opt import BayesianOptimization
# from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

NameError: name 'simplefilter' is not defined

### Load Data

In [12]:
PATH_DATA='D:/DSLS/Data Science/data/raw/'
filename_store='/preprocessed_dataset.pkl'
filename_valid='/validation_dataset.pkl'
filename_rec = '/recurring_dataset.pkl'

with open(PATH_DATA+filename_store, 'rb') as f:
  df = pickle.load(f)

with open(PATH_DATA+filename_valid, 'rb') as f:
  df_val = pickle.load(f)

with open(PATH_DATA+filename_rec, 'rb') as f:
  df_rec = pickle.load(f)

In [34]:
streets = ['Tol Jagorawi',
 'Tol Cijago',
 'Tol Depok-Antasari',
 'Kranggan Raya',
 'Tol Cimanggis-Cibitung',
 'Lenteng Agung Barat',
#  'N6 Cinangka Raya',
#  'Lenteng Agung Timur',
#  'Margonda Raya',
#  'N6 Jalan Raya Parung-Bogor'
          ]

In [35]:
samples=list()

for street in streets:
    for d in range(6):
        samples.append(df[street][d])
        print(samples)
        print()

[    time_on_day  median_delay  median_length  median_speed_kmh
0             1    159.500000    7146.500000         57.065002
1             2    174.750000    5857.750000         50.185001
2             3    135.500000    4429.000000         49.476667
3             5    324.340909    4645.454545         39.445000
4             6   1366.954545    7644.272727         17.223636
5             7    530.477273    4741.613636         33.305909
6             8     95.750000    2170.500000         39.227500
7             9    280.750000    2445.750000         30.887500
8            10    355.750000    2428.250000         30.811250
9            11    409.250000    2665.750000         30.541250
10           12    393.625000    2827.000000         28.720000
11           13    861.750000    4837.500000         16.280000
12           14    626.750000    3604.000000         16.757500
13           15    116.666667    1739.333333         35.518334
14           16    110.500000    1675.333333         3

[    time_on_day  median_delay  median_length  median_speed_kmh
0             1    159.500000    7146.500000         57.065002
1             2    174.750000    5857.750000         50.185001
2             3    135.500000    4429.000000         49.476667
3             5    324.340909    4645.454545         39.445000
4             6   1366.954545    7644.272727         17.223636
5             7    530.477273    4741.613636         33.305909
6             8     95.750000    2170.500000         39.227500
7             9    280.750000    2445.750000         30.887500
8            10    355.750000    2428.250000         30.811250
9            11    409.250000    2665.750000         30.541250
10           12    393.625000    2827.000000         28.720000
11           13    861.750000    4837.500000         16.280000
12           14    626.750000    3604.000000         16.757500
13           15    116.666667    1739.333333         35.518334
14           16    110.500000    1675.333333         3

[    time_on_day  median_delay  median_length  median_speed_kmh
0             1    159.500000    7146.500000         57.065002
1             2    174.750000    5857.750000         50.185001
2             3    135.500000    4429.000000         49.476667
3             5    324.340909    4645.454545         39.445000
4             6   1366.954545    7644.272727         17.223636
5             7    530.477273    4741.613636         33.305909
6             8     95.750000    2170.500000         39.227500
7             9    280.750000    2445.750000         30.887500
8            10    355.750000    2428.250000         30.811250
9            11    409.250000    2665.750000         30.541250
10           12    393.625000    2827.000000         28.720000
11           13    861.750000    4837.500000         16.280000
12           14    626.750000    3604.000000         16.757500
13           15    116.666667    1739.333333         35.518334
14           16    110.500000    1675.333333         3

[    time_on_day  median_delay  median_length  median_speed_kmh
0             1    159.500000    7146.500000         57.065002
1             2    174.750000    5857.750000         50.185001
2             3    135.500000    4429.000000         49.476667
3             5    324.340909    4645.454545         39.445000
4             6   1366.954545    7644.272727         17.223636
5             7    530.477273    4741.613636         33.305909
6             8     95.750000    2170.500000         39.227500
7             9    280.750000    2445.750000         30.887500
8            10    355.750000    2428.250000         30.811250
9            11    409.250000    2665.750000         30.541250
10           12    393.625000    2827.000000         28.720000
11           13    861.750000    4837.500000         16.280000
12           14    626.750000    3604.000000         16.757500
13           15    116.666667    1739.333333         35.518334
14           16    110.500000    1675.333333         3

In [36]:
# try2, try3 = split_train_test(samples[0])
# try2['median_delay']

### Hyperparameter Tuning (GRID SEARCH)

In [37]:
def split_train_test(data):
    train_data = data[:int(0.9*(len(data)))]
    test_data = data[int(0.9*(len(data))):]
    
#     print(f"shape of train: {train_data.shape}")
#     print(f"shape of test: {test_data.shape}")
    
    return train_data, test_data

In [38]:
def black_box_arima(p,d,q,street):
    mape =  list()
    metric = 'median_delay'
    
    samples_f = df[street]
#     print(samples_f)
    for i in range(6):
        # splitting
        sample = samples_f[i]
        
        train, test = split_train_test(sample)
        
        try:
            data_train=train[metric]
            data_test=test[metric]
        except:
            print('failed splitting')
            continue
            
        # modeling 
        try:
            y = data_train
            ARIMAmodel = ARIMA(y, order = (int(p), int(d), int(q)))
            ARIMAmodel = ARIMAmodel.fit()
        except:
            print('failed modelling')
            continue 
            
        # predict
        y_pred = ARIMAmodel.get_forecast(len(data_test.index))
        y_pred_df = y_pred.conf_int(alpha = 0.1) 
        y_pred_df["Predictions"] = ARIMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
        y_pred_df.index = data_test.index
        y_pred_out = y_pred_df["Predictions"] 
        
        
        mape.append(mean_absolute_percentage_error(data_test.values, y_pred_df["Predictions"]))
        
    return mean(mape)

In [39]:
mape = black_box_arima(1,0,3,'Tol Jagorawi')

print(mape)
# print(data_test)

0.8626534087402395


In [40]:
param_bounds = {
    'p':[i for i in range(4)],
    'd':[i for i in range(2)],
    'q':[i for i in range(10)]
}

In [41]:
print(f"  p  |  d  |  q  |  mape")
print(f"-------------------------------------------")

# for street in streets:
#     print(f"TUNING FOR STREET: {street}")
for p in param_bounds['p']:
    for d in param_bounds['d']:
        for q in param_bounds['q']:
            try:
                mape = black_box_arima(p,d,q,'Lenteng Agung Barat')
            except:
                mape ='Failed'
                continue
            if mape < 0.2:
                break

            print(f"  {p}  |  {d}  |  {q}  |  {mape}")

  p  |  d  |  q  |  mape
-------------------------------------------
  0  |  0  |  0  |  1.0972697906597726
  0  |  0  |  1  |  0.8787181572295595
  0  |  0  |  2  |  0.931603033340019
  0  |  0  |  3  |  0.9302437984159464
  0  |  0  |  6  |  1.4715534910502184
  0  |  0  |  7  |  1.6250154128330494
  0  |  0  |  8  |  1.3904958160884024
  0  |  0  |  9  |  1.4414475527443908
  1  |  0  |  0  |  0.6977852172789921
  1  |  0  |  1  |  0.8083780255913704
  1  |  0  |  2  |  0.9573912433671715
  1  |  0  |  3  |  1.2976002547070302
failed modelling
failed modelling
  1  |  0  |  4  |  0.7349378389925024
failed modelling
failed modelling
failed modelling
  1  |  0  |  5  |  1.7598874171261298
  1  |  0  |  6  |  1.6407272771968135
  1  |  0  |  7  |  1.594643055370615
  1  |  0  |  8  |  1.2839444013932642
  1  |  0  |  9  |  1.4763195434780478
  1  |  1  |  0  |  0.20208840593589153
  1  |  1  |  1  |  0.3917040277248861
  1  |  1  |  2  |  0.6406736548552968
failed modelling
  1  |  1  

### Results

In [46]:
params = {
    'Tol Jagorawi': (2,1,5),
    'Tol Cijago': (0,1,0),
    'Tol Depok-Antasari': (0,0,1),
    'Kranggan Raya':(2,1,2),
    'Tol Cimanggis-Cibitung':(2,1,9),
    'Lenteng Agung Barat':(2,1,0)
}

In [47]:
PATH_DATA='D:/DSLS/Data Science/data/raw/'
filename_params='/params_tuned.pkl'

In [48]:
with open(PATH_DATA+filename_params, 'wb') as f:
  pickle.dump(params,f)